In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

def load_images_and_masks(img_folder, mask_folder, img_size=(256, 256)):
    # Filter out .txt files or any unwanted files
    img_filenames = [f for f in sorted(os.listdir(img_folder)) if not f.endswith('.txt')]
    mask_filenames = [f for f in sorted(os.listdir(mask_folder)) if not f.endswith('.txt')]
    
    images = []
    masks = []
    
    for img_filename, mask_filename in zip(img_filenames, mask_filenames):
        img_path = os.path.join(img_folder, img_filename)
        mask_path = os.path.join(mask_folder, mask_filename)
        
        # Load image and mask
        img = load_img(img_path, target_size=img_size)
        img = img_to_array(img) / 255.0
        mask = load_img(mask_path, target_size=img_size, color_mode="grayscale")
        mask = img_to_array(mask) / 255.0
        
        images.append(img)
        masks.append(mask)
    
    return np.array(images), np.array(masks)

# Paths to the dataset
train_img_folder_path = "/kaggle/input/isic-2018/data/images/train"
train_mask_folder_path = "/kaggle/input/isic-2018/data/annotations/train"
val_img_folder_path = "/kaggle/input/isic-2018/data/images/val"
val_mask_folder_path = "/kaggle/input/isic-2018/data/annotations/val"
test_img_folder_path = "/kaggle/input/isic-2018/data/images/test"
test_mask_folder_path = "/kaggle/input/isic-2018/data/annotations/test"

# Load datasets
train_imgs, train_masks = load_images_and_masks(train_img_folder_path, train_mask_folder_path)
val_imgs, val_masks = load_images_and_masks(val_img_folder_path, val_mask_folder_path)
test_imgs, test_masks = load_images_and_masks(test_img_folder_path, test_mask_folder_path)

# Build the enhanced U-Net model
def conv_block(input_tensor, num_filters):
    x = layers.Conv2D(num_filters, (3, 3), padding="same", kernel_initializer="he_normal")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (3, 3), padding="same", kernel_initializer="he_normal")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def residual_block(input_tensor, num_filters):
    conv = conv_block(input_tensor, num_filters)
    res = layers.Conv2D(num_filters, (1, 1), padding="same")(input_tensor)
    output = layers.add([conv, res])
    return output

def attention_block(g, x, num_filters):
    g1 = layers.Conv2D(num_filters, (1, 1), padding="same")(g)
    g1 = layers.BatchNormalization()(g1)
    x1 = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    x1 = layers.BatchNormalization()(x1)
    psi = layers.Activation("relu")(layers.add([g1, x1]))
    psi = layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(psi)
    return layers.multiply([x, psi])

def build_enhanced_unet(input_shape=(256, 256, 3), num_classes=1):
    inputs = layers.Input(input_shape)

    # Encoder
    f1 = residual_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(f1)
    p1 = layers.Dropout(0.1)(p1)

    f2 = residual_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(f2)
    p2 = layers.Dropout(0.1)(p2)

    f3 = residual_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(f3)
    p3 = layers.Dropout(0.2)(p3)

    f4 = residual_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(f4)
    p4 = layers.Dropout(0.2)(p4)

    # Bottleneck
    bottleneck = residual_block(p4, 1024)
    bottleneck = layers.Dropout(0.3)(bottleneck)

    # Decoder
    u4 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding="same")(bottleneck)
    a4 = attention_block(u4, f4, 512)
    c4 = residual_block(a4, 512)

    u3 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding="same")(c4)
    a3 = attention_block(u3, f3, 256)
    c3 = residual_block(a3, 256)

    u2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(c3)
    a2 = attention_block(u2, f2, 128)
    c2 = residual_block(a2, 128)

    u1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(c2)
    a1 = attention_block(u1, f1, 64)
    c1 = residual_block(a1, 64)

    outputs = layers.Conv2D(num_classes, (1, 1), activation="sigmoid")(c1)

    model = models.Model(inputs, outputs)
    return model

model = build_enhanced_unet(input_shape=(256, 256, 3), num_classes=1)


model.summary()



In [ ]:
# Compile and train the model
model = build_enhanced_unet(input_shape=(256, 256, 3))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="binary_crossentropy",
              metrics=["accuracy", tf.keras.metrics.MeanIoU(num_classes=2)])

checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", mode="min")
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = model.fit(
    train_imgs, train_masks,
    validation_data=(val_imgs, val_masks),
    epochs=50,
    batch_size=16,
    callbacks=[checkpoint, early_stopping]
)

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy, test_miou = model.evaluate(test_imgs, test_masks, batch_size=16)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Mean IoU: {test_miou:.4f}")

# Predict on test images
predictions = model.predict(test_imgs)


In [ ]:
import matplotlib.pyplot as plt

# Function to display original image, ground truth mask, and predicted mask
def display_predictions(images, masks, predictions, num_samples=5):
    for i in range(num_samples):
        plt.figure(figsize=(12, 4))
        
        # Original image
        plt.subplot(1, 3, 1)
        plt.imshow(images[i])
        plt.title("Original Image")
        plt.axis("off")
        
        # Ground truth mask
        plt.subplot(1, 3, 2)
        plt.imshow(masks[i].squeeze(), cmap="gray")
        plt.title("Ground Truth Mask")
        plt.axis("off")
        
        # Predicted mask
        plt.subplot(1, 3, 3)
        plt.imshow(predictions[i].squeeze(), cmap="gray")
        plt.title("Predicted Mask")
        plt.axis("off")
        
        plt.show()

# Generate predictions
predictions = model.predict(test_imgs)
predictions = (predictions > 0.5).astype(np.uint8)  # Apply threshold to get binary masks

# Display predictions
display_predictions(test_imgs, test_masks, predictions, num_samples=5)
